Instalação das bilbiotecas
=================

In [1]:
!pip install mediapipe opencv-python

Importação das bibliotecas
=================

In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Definição do método da distância do tronco
=================

In [87]:
def dist_upper_body(shoulder,hip):
    shoulder = np.array(shoulder)
    hip = np.array(hip)
    
    distance = abs(((shoulder[1] - hip[1])/2)) * 80 
    
    return distance

Reconhecimento das medidas
=================

In [95]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
stage = None
header = None
second_header = None
third_header = None
color  = 0

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            hip_visibility = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility]
            width_shoulder = (landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x - landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x) * 200            
            width_hip = (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x - landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x) * 240
            
            
            #Size of upper body
            size_body = dist_upper_body(shoulder, hip)
            
            #Teste de distância
            if (hip_visibility[0] > 1 or hip_visibility[0] < 0.8) and (hip_visibility[1] > 1 or hip_visibility[1] < 0.8):
                header = 'Se afaste'
                second_header = None
                third_header = ""
                color = (17, 17, 244) #vermelho
            else:
                header = "{}cm de ombro.".format(round(width_shoulder,0)) 
                second_header = "{}cm de largura da cintura.".format(round(width_hip,0))
                third_header = "{}cm tamanho do tronco.".format(round(size_body,0))                
                color = (66, 191, 79) #verde
                                
        except:
            pass
        
                            #1 vert   2 vert    BGR
        cv2.rectangle(image, (640,65), (0,0), color, -1)
        
        # Stage data
        cv2.putText(image, 'Status', (60,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, header, (60,30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, second_header, (60,60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(third_header), (320,30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Detector de Medida', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    #Marcos
    #42cm de ombro
    #49cm do ombro até a cintura
    #31cm da largura da cintura